# Исследование надежности заемщиков


## Откроем таблицу и изучим общую информацию о данных

**Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраним в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Выведем первые 20 строчек датафрейма `data` на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Выведем количество пропущенных значений для каждого столбца. Используем комбинацию двух методов.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получились аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится нам для исследования.

**Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Заполним пропуски в столбце `days_employed` медианными значениями каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверим остальные столбцы.**

In [14]:
data['education'] = data['education'].str.lower()

**Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут вам правильно определить категорию.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследуем данные и ответим на вопросы

#### 1. Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
children_table = data.groupby(['children']).agg({'children':'count', 'debt':'sum'}) #создаем таблицу, в которой группируем данные по параметру "количество детей в семье", столбцы в новой таблице:
                                                                                    # - количество кредитов в каждой из категорий (столбец 'children', который в следующей строке будет переименован на 'credit_counts'), 
                                                                                    # - количество задолженностей по возврату кредитов в каждой из категории (столбец 'debt')
children_table = children_table.rename (columns = {'children' : 'credit_counts'}) #переименовываем столбец с общим количеством данных по каждой из категорий ('children' в 'credit_counts')
children_table['conversion'] = children_table['debt']/children_table['credit_counts'] #рассчитаем отношение задолженностей по кредиту к общему числу кредитов в каждой категории (создаем столбец 'conversation')
children_table.sort_values(by='conversion', ascending = False) #вывод данных для анализа (для наглядности данные отсортированы по столбцу 'conversation' по убыванию)

,credit_counts,debt,conversion
children,,,
4,41,4,0.097561
2,2052,194,0.094542
1,4808,444,0.092346
3,330,27,0.081818
0,14091,1063,0.075438
5,9,0,0.000000


**Вывод:** 
Из полученных данных видно, что нет **прямой** зависимости между количеством детей и возвратом кредита в срок. При этом, если не учитывать данные о семьях с 5 детьми (их всего 9 из 21331), то можно сделать вывод, что семьи без детей реже имеют задолженность по кредиту, чем семьи, имеющие хотя бы одного ребенка.

In [24]:
def categorize_children_count(children): #функция на основании данных столбца 'children' создает столбец с категорией "количество детей", объединяя категории с 3, 4, 5 детьми в категорию "3+".
    try:
        if children == 0:
            return 'нет детей'
        elif children == 1:
            return 'один ребенок'
        elif children == 2:
            return 'двое детей'
        elif children >= 3:
            return 'больше двух детей'
    except:
        pass
data['children_category'] = data['children'].apply(categorize_children_count)
    
def df (parameter):
    df = data.groupby(parameter).agg({parameter:'count', 'debt':'sum'}) #создаем таблицу, в которой группируем данные по параметру "parameter", столбцы в новой таблице:
                                                                                    # - количество кредитов в каждой из категорий (столбец, который в следующей строке будет переименован на 'credit_counts'), 
                                                                                    # - количество задолженностей по возврату кредитов в каждой из категории (столбец 'debt')
    df = df.rename (columns = {parameter : 'credit_counts'}) #переименовываем столбец с общим количеством данных по каждой из категорий (в 'credit_counts')
    df['conversion'] = df['debt']/df['credit_counts'] #рассчитаем отношение задолженностей по кредиту к общему числу кредитов в каждой категории (создаем столбец 'conversation')
    df = df.sort_values(by='conversion', ascending = False) #вывод данных для анализа (для наглядности данные отсортированы по столбцу 'conversation' по убыванию)
    return df
children_table = df ('children_category')
children_table

,credit_counts,debt,conversion
children_category,,,
двое детей,2052,194,0.094542
один ребенок,4808,444,0.092346
больше двух детей,380,31,0.081579
нет детей,14091,1063,0.075438


#### 2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
family_status_table = data.groupby(['family_status']).agg({'family_status':'count', 'debt':'sum'}) #создаем таблицу, в которой группируем данные по параметру "семейное положение", столбцы в новой таблице:
                                                                                                   # - количество кредитов в каждой из категорий (столбец 'family_status', который в следующей строке будет переименован на 'credit_counts'), 
                                                                                                   # - количество задолженностей по возврату кредитов в каждой из категории (столбец 'debt')
family_status_table = family_status_table.rename (columns = {'family_status' : 'credit_counts'}) #переименовываем столбец с общим количеством данных по каждой из категорий ('family_status' в 'credit_counts')
family_status_table['conversion'] = family_status_table['debt']/family_status_table['credit_counts'] #рассчитаем отношение задолженностей по кредиту к общему числу кредитов в каждой категории (создаем столбец 'conversation')
family_status_table.sort_values(by='conversion', ascending = False) #вывод данных для анализа (для наглядности данные отсортированы по столбцу 'conversation' по убыванию)

,credit_counts,debt,conversion
family_status,,,
Не женат / не замужем,2796,273,0.097639
гражданский брак,4134,385,0.093130
женат / замужем,12261,927,0.075606
в разводе,1189,84,0.070648
вдовец / вдова,951,63,0.066246


**Вывод:** 
Основываясь на полученные данные, можно предположить, что незамужние/неженатые клиенты банка чаще позволяют себе задолженность по кредиту, чем те, кто ранее (речь о статусах "в разводе" и "вдовец/вдова") или в настоящий момент ("гражданский брак", "женат/замужем") находился/находится в браке (официальном или гражданском).

#### 3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [28]:
total_income_category_table = data.groupby(['total_income_category']).agg({'total_income_category':'count', 'debt':'sum'}) #создаем таблицу, в которой группируем данные по параметру "уровень дохода", столбцы в новой таблице:
                                                                                                                           # - количество кредитов в каждой из категорий (столбец 'total_income_category', который в следующей строке будет переименован на 'credit_counts'), 
                                                                                                                           # - количество задолженностей по возврату кредитов в каждой из категории (столбец 'debt')
total_income_category_table = total_income_category_table.rename (columns = {'total_income_category' : 'credit_counts'}) #переименовываем столбец с общим количеством данных по каждой из категорий ('total_income_category' в 'credit_counts')
total_income_category_table['conversion'] = total_income_category_table['debt']/total_income_category_table['credit_counts'] #рассчитаем отношение задолженностей по кредиту к общему числу кредитов в каждой категории (создаем столбец 'conversation')
total_income_category_table #вывод данных для анализа

,credit_counts,debt,conversion
total_income_category,,,
A,25,2,0.080000
B,5014,354,0.070602
C,15921,1353,0.084982
D,349,21,0.060172
E,22,2,0.090909


**Вывод:** 
Казалось бы, уровень дохода должен напрямую влиять на параметр "задолженность по кредиту", но из полученных данных видно, что прямой зависимости между вышеуказанными показателями нет.
Для наглядности скопируем данные по категориям уровня дохода, которые были применены в задании 16:
- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

Но, возможно, полученные данные некорректно отражают общую картину из-за:
 - некорректного категорирования доходов (выбранные интервалы доходов неодинаковые);
 - искажения данных при заполнении пропусков (именно в этом столбце были выявлены пропуски в исходных данных).

#### 4. Как разные цели кредита влияют на его возврат в срок?

In [30]:
purpose_category_table = data.groupby(['purpose_category']).agg({'purpose_category':'count', 'debt':'sum'}) #создаем таблицу, в которой группируем данные по категориям "цели кредита", столбцы в новой таблице:
                                                                                                            # - количество кредитов в каждой из категорий (столбец 'purpose_category', который в следующей строке будет переименован на 'credit_counts'), 
                                                                                                            # - количество задолженностей по возврату кредитов в каждой из категории (столбец 'debt')
purpose_category_table = purpose_category_table.rename (columns = {'purpose_category' : 'credit_counts'}) #переименовываем столбец с общим количеством данных по каждой из категорий ('purpose_category' в 'credit_counts')
purpose_category_table['conversion'] = purpose_category_table['debt']/purpose_category_table['credit_counts'] #рассчитаем отношение задолженностей по кредиту к общему числу кредитов в каждой категории (создаем столбец 'conversation')
purpose_category_table.sort_values(by='conversion', ascending = False) #вывод данных для анализа (для наглядности данные отсортированы по столбцу 'conversation' по убыванию)

,credit_counts,debt,conversion
purpose_category,,,
операции с автомобилем,4279,400,0.093480
получение образования,3988,369,0.092528
проведение свадьбы,2313,183,0.079118
операции с недвижимостью,10751,780,0.072551


**Вывод:** 
Кредиты, выдаваемые на операции с недвижимостью, реже попадают под категорию наличия задолженности, при этом кредиты, выдаваемые на приобретение автомобиля или образования с бОльшей долей вероятности могут быть просрочены.

#### 5. Приведем возможные причины появления пропусков в исходных данных.


Если говорить о появлении пропусков в исходных данных в целом, то можно указать такие причины, как:
* человеческий фактор (ошибки при вводе данных);
* ошибки при чтении/копировании/форматировании файла;
* отсутствие информации в принципе и др.

**Посмотрим отдельно на данные в нашем проекте.**

Пропуски присутствуют в столбцах с данными об **общем трудовом** стаже и **ежемесячном доходе**. При этом количество ненулевых значений в этих столбцах совпадает (19351 из 21525, данные приведены из результатов выполнения задания 3).
Первым предположением было то, что данные о стаже и доходе могут быть не заполнены, например, у студентов (так как они еще не работали и/или не работают), но выбрав строки с пропущенными данными и сгруппировав их по типу занятости, были получены следующие значения:

*для столбца со стажем работы:*       
* сотрудник - 1105                 
* компаньон - 508                  
* пенсионер - 413                                    
* госслужащий - 147                                
* предприниматель - 1                                

*для столбца с данными о доходах:*
* сотрудник - 1105                               
* компаньон - 508                                
* пенсионер - 413                                
* госслужащий - 147                               
* предприниматель - 1 

То есть вышеуказанное предположение не подтвердилось.


Обратите внимание, что данные в двух указанных выше выборках совпали. При этом выше отмечено, что количество ненулевых значений в рассматриваемых нами столбцах (общий трудовой стаж и данные о ежемесячном доходе) также совпадает.
Посчитав количество строк, в которых нулевые значения встречаются одновременно в обоих столбцах, получим 2174 строки (2174 + 19351 = 21525), то есть пропуски данных об общем трудовом стаже и ежемесячном доходе зафиксированы в одних и тех же строках.

Вывод: вышеуказанное изучение исходных данных позволяет сделать предположение, что на каком-то этапе сбора данных **информация о стаже работы и ежемесячном доходе у клиента не запрашивалась или хранилась в отдельном файле и он был утерян**.

#### 6. Объясним, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* 
Среднее арифметическое и медиана оценивают среднее значение выборки. Среднее арифметическое некорректно применять при наличии данных, которые намного больше и/или меньше бОльшего количества значений, так как среднее значение при таких данных будет смещено в бОльшую или меньшую сторону.
Вот почему корректнее применять медианное значение при нахождении средних параметров выборки.

### Общий вывод.

При рассмотрении заявки на выдачу кредита есть необходимость прибегнуть к анализу статистики по ранее выданным кредитам, чтобы снизить вероятность задолженностей по кредитам.
Из проведенного анализа исходных данных можно сделать следующие выводы:
 - нет **прямой** зависимости между количеством детей и возвратом кредита в срок, но при этом, если не учитывать данные о семьях с 5 детьми (их всего 9 из 21331), то можно сделать вывод, что семьи без детей реже имеют задолженность по кредиту, чем семьи, имеющие хотя бы одного ребенка;
 - можно предположить, что незамужние/неженатые клиенты банка чаще позволяют себе задолженность по кредиту, чем те, кто ранее (речь о статусах "в разводе" и "вдовец/вдова") или в настоящий момент ("гражданский брак", "женат/замужем") находился/находится в браке (официальном или гражданском);
 - для получения данных о зависимости между уровнем дохода и возвратом кредита в срок, вероятно, следует пересмотреть критерии исходных данных;
 - кредиты, выдаваемые на операции с недвижимостью, реже попадают под категорию наличия задолженности, при этом кредиты, выдаваемые на приобретение автомобиля или образования с бОльшей долей вероятности могут быть просрочены.